---
# Running GS 
---

- Main Purpose : use the functions defined in Equations.py and run a grid search
- functions are imported -> parameter grid is defined -> code to store files is written 

- Code is in the first hald of the notebook , beyond "TESTING BEYOND THIS CELL" is just random testing 

In [44]:
import pandas as pd
import numpy
import importlib
import itertools

from Equations_Run_Combo_V_2 import *

In [45]:
# cc = {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 15, 'batch_size': 50, 'use_bidirectional': False, 'lag': 3, 'input_size': 12, 'hidden_size': 35,
#       'num_layers': 6, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': True, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 
#       'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 
#       'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 36898, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False,
#       'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.5, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 
#       'use_USO_wticoncat_predictor_WEEKLY_END_MO': False, 'use_UCO_wticoncat_predictor_WEEKLY_END_MO': False, 'use_HUC_wticoncat_predictor_WEEKLY_END_MO': False, 'use_HOD_wticoncat_predictor_WEEKLY_END_MO': True,
#       'use_CRUD_wticoncat_predictor_WEEKLY_END_MO': False, 'use_SCO_wticoncat_predictor_WEEKLY_END_MO': False, 'train_start_month': '2004-01', 'val_start_month': '2020-01', 'val_end_month': '2021-12', 
#       'num_preds_per_fold': 3}

# r = run_combo_V_4(0, cc, 4, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)


In [46]:
###                             FIRST

import math 
from joblib import parallel_backend , Parallel , delayed 
import random 


######    GS 


param_grid = {
                      "binary_0_1_cutoff_ret_rate_percentage" : [0.1],
    "learning_rate": [0.05, 0.005, 0.0005 , 0.00005],
    "num_epochs": [ 70, 150, 300, 700],
    "batch_size": [10, 30, 50, 70],
    "use_bidirectional": [False],
    "lag": [ 2, 3, 6 , 8],
    "input_size": [12],
    "hidden_size": [ 12, 20, 35, 55],
    "num_layers": [1, 2, 4 , 6],
                                            "use_monthly_dfs_only": [True],
                                            
                                            "use_binary_0_1_retRate": [False],

                "use_custom_loss_function_BCE_THRESH": [False , True], # NEW NEW NEW
                "use_custom_loss_function_BCE_THRESH_AND_SEVERITY": [False, True], # NEW NEW NEW
                
                
                "use_LOW_weights_for_BCE_custom_loss": [True], # NEW NEW NEW
                "pred_threshold_sigmoid01_up": [None], # NEW NEW NEW this is applied later ... 


    "use_binary_neg1_1": [False],  
    "use_ret_rate": [False],       
    "use_print_acc": [False],      
    "use_dropout": [False],  
    # "iter_per_valSET": [1],        
                                             "use_class_weighting": [True, False],

    "is_deterministic": [True],     "seed_num": [42],       
    "use_existing_lagged_data": [True],'use_dynamic_weights' : [False] ,


    'use_binary_0_1_retRate_custom_neg' : [False] ,
    'use_binary_0_1_retRate_custom_pos' : [True] ,
    

    'POS_weight_multiplier' : [ .7 ,  1  ,  1.3  ] , 
    'use_rolling_fixed_train_size' : [False],
    'use_existing_initial_weights': [False], 
    'state_dict' : [None],

    "use_USO_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_UCO_wticoncat_predictor_WEEKLY_END_MO" : [False] , ### NEW NEW
    "use_HUC_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_HOD_wticoncat_predictor_WEEKLY_END_MO" : [True] , ### HOD 
    "use_CRUD_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_SCO_wticoncat_predictor_WEEKLY_END_MO" : [False] ,


                    'train_start_month'  : ["2004-01" , "2008-01"],  ### the 2004 start month includes data past the HOD inception date of 2008, the data is the adjusted wti price data
                    'val_start_month'  : ['2021-01'] ,    # test_start_month = '2022-01' ; test_end_month = '2022-12' 
                    'val_end_month'  : ['2022-12'] ,
                    'num_preds_per_fold' : [3] , 

  } 

# --- Prepare all combinations ---
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

#### remove repeats when custom loss is true  - this happends if there is a custom loss fucntion adn a POS weight multiplier , so remove all but one 
#### of the POS weight multipliers if a custom loss function is used
### ALSO remove all with use_class_weighting = False  since there are repeats for true adn false when custom loss function is used
### SO jsut keep when POS_weight_multiplier = 1 if custom loss = True to avoid repreats
# ALSO remove all with both custom loss functions = True since they are repeats

print(f"Total combinations before removing repeats: {len(combinations)}")

### run in parrallel or too time consuming

def remove_items_with_condition(combos):

    for i in combos[:]:

        if (i['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] == True) and (i['use_custom_loss_function_BCE_THRESH'] == True) :
            combos.remove(i)
        
        elif (i['use_custom_loss_function_BCE_THRESH'] == True) :
            if (i['POS_weight_multiplier'] != 1) and (i['use_class_weighting'] != True) :
                combos.remove(i)
    
        elif (i['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] == True) :
            if (i['POS_weight_multiplier'] != 1) and (i['use_class_weighting'] != True) :
                combos.remove(i)

    return combos


splits  = np.linspace(0, len(combinations), num=8, dtype=int).tolist()

for i in range(len(splits)-1):
    globals()[f"combos_part_{i}"] = combinations[splits[i]:splits[i+1]]

with parallel_backend("loky", n_jobs=-1):
    res = Parallel()(
        delayed(remove_items_with_condition)(globals()[f"combos_part_{i}"])
        for i in range(len(splits)-1)
    )


combos_concat_HOD = []
for i in res:
    combos_concat_HOD.extend(i)

print(f"Total combinations after removing repeats: {len(combos_concat_HOD)}")


combos_concat_UCO = []
for combo in copy.deepcopy(combos_concat_HOD):
    combo["use_UCO_wticoncat_predictor_WEEKLY_END_MO"] = True
    combo["use_HOD_wticoncat_predictor_WEEKLY_END_MO"] = False
    combos_concat_UCO.append(combo)


# combos_concat_UCO[0]

Total combinations before removing repeats: 196608
Total combinations after removing repeats: 114688


In [ ]:


def random_subset(data,  seed=42):
    random.seed(seed)
    n_keep = int(len(data))
    return random.sample(data, n_keep)


# --- Split combinations into chunks ---
def split_into_chunks(lst, n_chunks):
    chunk_size = len(lst) // n_chunks
    chunks = []
    for i in range(n_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < n_chunks - 1 else len(lst)
        chunks.append(lst[start:end])
    return chunks


random_subset_combos_HOD = random_subset(combos_concat_HOD, seed=42)

random_subset_combos_UCO = random_subset(combos_concat_UCO, seed=42)


# FRACTION_TO_KEEP = 1  ; fraction_idx = int(len(random_subset_combos) * FRACTION_TO_KEEP)


n_chunks = 21

combo_chunks_HOD = split_into_chunks(random_subset_combos_HOD[:], n_chunks)
combo_chunks_UCO = split_into_chunks(random_subset_combos_UCO[:], n_chunks)

# combo_chunks = split_into_chunks(combinations, n_chunks)

chunk_to_run_HOD = 0 #          <<------- 
chunk_to_run_UCO = 0 #          <<-------

selected_chunk_HOD = combo_chunks_HOD[chunk_to_run_HOD]
selected_chunk_UCO = combo_chunks_UCO[chunk_to_run_UCO]


In [ ]:
len(selected_chunk_UCO)

selected_chunk_UCO_copy_reduced = copy.deepcopy(selected_chunk_UCO)

counter = 0

for i in range(3000):

    if selected_chunk_UCO_copy_reduced[i]['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] != True:
        counter += 1
        del selected_chunk_UCO_copy_reduced[i]

    if counter >= 4411 - (4411- 3000):
        break



In [103]:
len(selected_chunk_UCO_copy_reduced)

# len([i for i in selected_chunk_UCO_copy_reduced if i['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] == True])



selected_chunk_UCO_copy_reduced[0]

{'binary_0_1_cutoff_ret_rate_percentage': 0.1,
 'learning_rate': 0.05,
 'num_epochs': 300,
 'batch_size': 10,
 'use_bidirectional': False,
 'lag': 2,
 'input_size': 12,
 'hidden_size': 35,
 'num_layers': 2,
 'use_monthly_dfs_only': True,
 'use_binary_0_1_retRate': False,
 'use_custom_loss_function_BCE_THRESH': False,
 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False,
 'use_LOW_weights_for_BCE_custom_loss': True,
 'pred_threshold_sigmoid01_up': None,
 'use_binary_neg1_1': False,
 'use_ret_rate': False,
 'use_print_acc': False,
 'use_dropout': False,
 'use_class_weighting': True,
 'is_deterministic': True,
 'seed_num': 42,
 'use_existing_lagged_data': True,
 'use_dynamic_weights': False,
 'use_binary_0_1_retRate_custom_neg': False,
 'use_binary_0_1_retRate_custom_pos': True,
 'POS_weight_multiplier': 1.3,
 'use_rolling_fixed_train_size': False,
 'use_existing_initial_weights': False,
 'state_dict': None,
 'use_USO_wticoncat_predictor_WEEKLY_END_MO': False,
 'use_UCO_wticoncat_pr

In [96]:
with open('selected_chunk_UCO_21_01__22_12_GS_____NEW.pkl', 'wb') as f:
    pickle.dump(selected_chunk_UCO_copy_reduced, f)

In [92]:
with open('selected_chunk_UCO_21_01__22_12_GS_____NEW.pkl', 'rb') as f:
    selected_chunk_UCO_copy_reduced = pickle.load(f)


with parallel_backend("loky", n_jobs=-1):
    results_HOD = Parallel()(
        delayed(run_combo_V_4)(i, combo, 0 , use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(selected_chunk_UCO_copy_reduced)
    )

#HERE

KeyboardInterrupt: 

In [18]:
len([combo for combo in selected_chunk_HOD if combo['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] == True and combo['use_custom_loss_function_BCE_THRESH'] == True])
len([combo for combo in selected_chunk_HOD if combo['use_HOD_wticoncat_predictor_WEEKLY_END_MO'] == True])


selected_chunk_UCO_BCE_SEV = []
for i in copy.deepcopy(selected_chunk_HOD):
    if i['use_custom_loss_function_BCE_THRESH_AND_SEVERITY'] == True:
        selected_chunk_UCO_BCE_SEV.append(i)

len(selected_chunk_UCO_BCE_SEV)


# with parallel_backend("loky", n_jobs=-1):
#     results_HOD = Parallel()(
#         delayed(run_combo_V_4)(i, combo, 6000 , use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
#         for i, combo in enumerate(selected_chunk_UCO_BCE_SEV)
#     )

1060

In [21]:
# with parallel_backend("loky", n_jobs=-1):
#     results_HOD = Parallel()(
#         delayed(run_combo_V_4)(i, combo, 6000 , use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
#         for i, combo in enumerate(selected_chunk_UCO_BCE_SEV[:500])
#     )

In [15]:

print(f"Running chunk {chunk_to_run_HOD+1} of {n_chunks} ({len(selected_chunk_HOD)} combos)")

# --- Calculate offset ---
total_offset_HOD = sum(len(chunk) for chunk in combo_chunks_HOD[:chunk_to_run_HOD])

# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results_HOD = Parallel()(
        delayed(run_combo_V_4)(i, combo, total_offset_HOD, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(selected_chunk_HOD)
    )

# --- Unzip results ---
metrics_list_HOD, weights_list_HOD = zip(*results_HOD)

Running chunk 1 of 32 (3584 combos)


--- Running Combo 7 ---Parameters: {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 70, 'batch_size': 10, 'use_bidirectional': False, 'lag': 3, 'input_size': 12, 'hidden_size': 20, 'num_layers': 1, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': True, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 'use_USO_wticoncat_predictor_WEEKLY_END_MO': False, 'use_UCO_wticoncat

KeyboardInterrupt: 

In [ ]:

print(f"Running chunk {chunk_to_run_UCO+1} of {n_chunks} ({len(selected_chunk_UCO)} combos)")

# --- Calculate offset ---
total_offset_UCO = sum(len(chunk) for chunk in combo_chunks_UCO[:chunk_to_run_UCO])

# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results_UCO = Parallel()(
        delayed(run_combo_V_4)(i, combo, total_offset_UCO, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False , store_model_weights=False)
        for i, combo in enumerate(selected_chunk_UCO)
    )

# --- Unzip results ---
metrics_list_UCO, weights_list_UCO = zip(*results_UCO)

Running chunk 1 of 20 (6553 combos)
--- Running Combo 5 ---Parameters: {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 300, 'batch_size': 30, 'use_bidirectional': False, 'lag': 2, 'input_size': 12, 'hidden_size': 55, 'num_layers': 1, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': False, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 'use_USO_wticoncat_predictor_WEE

KeyboardInterrupt: 

In [ ]:
import json


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.integer):
            return int(obj)
        elif isinstance(obj, numpy.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        else:
            return super(NpEncoder, self).default(obj)


file_json_metrics_HOD = f"GS_21_01_to_22_12_chunk_{chunk_to_run_HOD + 1}_{n_chunks}_10percentPOS_Vset_HOD.json"
# file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

# file_weights_HOD = f"GS_20_01_to_21_12_chunk_{chunk_to_run_HOD + 1}_{n_chunks}__8fold_10percentPOS_Vset_HOD.pt"
# # file_weights = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_WEIGHTS_8fold_05NEG__VERSION_2.pt"



with open(file_json_metrics_HOD, "w") as f_json:
    json.dump(metrics_list_HOD, f_json, indent=2)



# weights_list = copy.deepcopy(weights_list)

# torch.save(weights_list, file_weights)        



In [ ]:
import json


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.integer):
            return int(obj)
        elif isinstance(obj, numpy.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.DatetimeIndex):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        else:
            return super(NpEncoder, self).default(obj)


file_json_metrics_UCO = f"GS_21_01_to_22_12_chunk_{chunk_to_run_UCO + 1}_{n_chunks}_10percentPOS_Vset_UCO.json"
# file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

# file_weights_UCO = f"GS_20_01_to_21_12_chunk_{chunk_to_run_UCO + 1}_{n_chunks}__8fold_10percentPOS_Vset_UCO.pt"
# # file_weights = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_WEIGHTS_8fold_05NEG__VERSION_2.pt"


with open(file_json_metrics_UCO, "w") as f_json:
    json.dump(metrics_list_UCO, f_json, indent=2)




In [ ]:
# class np_encoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, pd.Timestamp):
#             return obj.isoformat()
#         elif isinstance(obj, torch.Tensor):
#             return obj.tolist()
#         elif isinstance(obj, pd.DatetimeIndex):
#             return obj.tolist()
#         else:
#             return super(np_encoder, self).default(obj)


---
# TESTING BEYOND THIS CELL
---